In [21]:
import pandas as pd
from kats.models.prophet import ProphetModel, ProphetParams
# from kats.consts import TimeSeriesData
# # from kats.models.var import VARModel, VARParams
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
import numpy as np
import matplotlib.pyplot as plt
import os.path

ModuleNotFoundError: No module named 'kats'

In [20]:
MINIMAL_KATS=1 pip install kats

SyntaxError: invalid syntax (1417110850.py, line 1)

In [16]:
## podatki, ki so nam jih poslali:
podatki = pd.read_csv(os.path.join('C:/Users/Uporabnik/OneDrive/Namizje/SOLA/MAGISTERIJ/MATEMATIKA Z RACUNALNIKOM/GospodinjskiOdjemalci.csv'), sep=";")
# Malo boljši format:
podatki['DateTimeStartUTC'] = pd.to_datetime(podatki['DateTimeStartUTC'], format='%d.%m.%Y %H:%M')
podatki['DateTimeStartCET'] = pd.to_datetime(podatki['DateTimeStartCET'], format='%d.%m.%Y %H:%M')

podatki = podatki[['DateTimeStartUTC', 'Odjem ACT', 'Temperatura ACT', 'Temperatura FC', 'Sevanje ACT', 'Sevanje FC']]

podatki.set_index('DateTimeStartUTC', inplace=True)
podatki['Year'] = podatki.index.year
podatki['Month'] = podatki.index.month
podatki['Day'] = podatki.index.date

podatki = podatki.asfreq('15T')

podatki['Odjem ACT'] = podatki['Odjem ACT'] * 1000000000 # da dobim MWh (bo lepše pri logaritmu tega stolpce v nadaljevanju)

podatki

,Odjem ACT,Temperatura ACT,Temperatura FC,Sevanje ACT,Sevanje FC,Year,Month,Day
DateTimeStartUTC,,,,,,,,
2021-10-31 23:00:00,9508.67,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:15:00,9168.37,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:30:00,8860.11,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:45:00,8510.97,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-11-01 00:00:00,8332.21,3.45,5.300,0.0,0.0,2021,11,2021-11-01
...,...,...,...,...,...,...,...,...
2024-02-29 21:45:00,12005.50,6.80,7.475,0.0,0.0,2024,2,2024-02-29
2024-02-29 22:00:00,11336.40,6.65,7.300,0.0,0.0,2024,2,2024-02-29
2024-02-29 22:15:00,10796.70,6.50,7.300,0.0,0.0,2024,2,2024-02-29


In [ ]:
# multivariatni modeli

In [17]:
# da bo šlo bolj hitro se zaenkrat omejimo na samo na dva meseca:
# train data:
train_data = podatki['2023-02-25':'2023-02-28']
train_data_OG = podatki.loc['2023-02-22':'2023-02-28', 'Odjem ACT']
temp_data = podatki.loc['2023-02-22':'2023-02-28', 'Temperatura FC']

# test data:
test_data = podatki['2023-03-01']
test_data_OG = podatki.loc['2023-03-01', 'Odjem ACT']
test_temp_data = podatki.loc['2023-03-01', 'Temperatura FC']

C:\Users\Uporabnik\AppData\Local\Temp\ipykernel_16168\142242945.py:8: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  test_data = podatki['2023-03-01']


In [18]:
# 1. LMulti-Variate Time-series Forecasting using Kats Model

multi_ts = TimeSeriesData(train_data)
params = VARParams()
m = VARModel(multi_ts, params)
m.fit()

NameError: name 'TimeSeriesData' is not defined